In [ ]:
from google.colab import files
uploaded = files.upload()  # ← 여기서 messages_only.jsonl만 선택
DATA_FILE = list(uploaded.keys())[0]
print("✅ 업로드 완료:", DATA_FILE)

Saving 뒤지고싶냐.jsonl to 뒤지고싶냐.jsonl
✅ 업로드 완료: 뒤지고싶냐.jsonl


In [ ]:
import json, pandas as pd

rows = []
with open(DATA_FILE, "r", encoding="utf-8") as f:   # utf-8-sig가 필요하면 그걸로 바꿔
    for line in f:
        line = line.strip()
        if not line:
            continue
        rows.append(json.loads(line))

df = pd.DataFrame(rows)
assert {"message","label"}.issubset(df.columns), df.columns
print("✅ 로드:", df.shape)
df.head(5)

✅ 로드: (12660, 2)


,message,label
0,저렇게는 살지 말아야지,toxic
1,볼 게 없어서 벗방을 보냐 ㅋㅋㅋ,non_toxic
2,쉴드가 아니라 국가가 면제해준거야.,toxic
3,짜져라 이 돼지 꿀꿀아,toxic
4,짱깨는 반드시 해체되어야 함,toxic


In [ ]:
def normalize_label(x):
    if isinstance(x, str):
        xs = x.strip().lower()
        if xs in ["toxic","1","true","yes","y"]:
            return 1
        if xs in ["non_toxic","0","false","no","n","neutral","clean"]:
            return 0
    if isinstance(x, (int, float)):
        return 1 if int(x)==1 else 0
    return 0

df["label"] = df["label"].apply(normalize_label)
print("라벨 분포:\n", df["label"].value_counts())


라벨 분포:
 label
1    7254
0    5406
Name: count, dtype: int64


In [ ]:
from google.colab import files
uploaded_bad = files.upload()  # ← 여기서 bad_words.txt만 선택
BADWORDS_FILE = list(uploaded_bad.keys())[0]
print("✅ 업로드 완료:", BADWORDS_FILE)

Saving bad_words.txt to bad_words.txt
✅ 업로드 완료: bad_words.txt


In [ ]:
import re

# 1) 정규식 컴파일
patterns = []
with open(BADWORDS_FILE, "r", encoding="utf-8") as f:
    for line in f:
        raw = line.strip()
        if not raw or raw.startswith("#"):
            continue
        if raw.startswith("re:"):
            pat = raw[3:].strip()
        else:
            pat = re.escape(raw)
        patterns.append(pat)

BADWORDS_RE = re.compile("(?:" + "|".join(patterns) + ")", flags=re.IGNORECASE) if patterns else None

def contains_badword(s: str) -> bool:
    if BADWORDS_RE is None or not isinstance(s, str):
        return False
    return BADWORDS_RE.search(s) is not None


# 2) (옵션) 욕설 규칙 강제 적용
FORCE_BADWORDS_TOXIC = True  # 필요 없으면 False
if FORCE_BADWORDS_TOXIC and BADWORDS_RE is not None:
    has_bad = df["message"].apply(contains_badword)
    before = df["label"].sum()
    df.loc[has_bad, "label"] = 1
    after = df["label"].sum()
    print(f"⚙️ 욕설 규칙 적용: toxic {before} → {after} (+{after-before})")
else:
    print("욕설 규칙 미적용 또는 사전 없음")

⚙️ 욕설 규칙 적용: toxic 7254 → 7260 (+6)


In [ ]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)
print("train:", train_df.shape, "valid:", valid_df.shape)
print("train 분포:\n", train_df["label"].value_counts())
print("valid 분포:\n", valid_df["label"].value_counts())

train: (10128, 2) valid: (2532, 2)
train 분포:
 label
1    5808
0    4320
Name: count, dtype: int64
valid 분포:
 label
1    1452
0    1080
Name: count, dtype: int64


In [ ]:
!pip install -q -U transformers datasets accelerate evaluate scikit-learn

import torch, numpy as np, pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 135.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
MODEL_NAME = "klue/roberta-base"   # 가벼운 버전은 "klue/roberta-small"
MAX_LEN = 160

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tok_fn(batch):
    return tokenizer(
        batch["message"],            # ← message 컬럼 사용
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN
    )

# HF Datasets로 변환
train_ds = Dataset.from_pandas(train_df[["message","label"]].rename(columns={"message":"text","label":"labels"}), preserve_index=False)
valid_ds = Dataset.from_pandas(valid_df[["message","label"]].rename(columns={"message":"text","label":"labels"}), preserve_index=False)

# 토크나이즈
train_ds = train_ds.map(lambda b: tokenizer(b["text"], padding="max_length", truncation=True, max_length=MAX_LEN), batched=True, remove_columns=["text"])
valid_ds = valid_ds.map(lambda b: tokenizer(b["text"], padding="max_length", truncation=True, max_length=MAX_LEN), batched=True, remove_columns=["text"])

# 필요한 컬럼만 남기기
keep = ["input_ids","attention_mask","labels"]
train_ds = train_ds.remove_columns([c for c in train_ds.column_names if c not in keep])
valid_ds = valid_ds.remove_columns([c for c in valid_ds.column_names if c not in keep])

datasets = DatasetDict(train=train_ds, validation=valid_ds)
datasets

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Map:   0%|          | 0/10128 [00:00<?, ? examples/s]

Map:   0%|          | 0/2532 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 10128
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2532
    })
})

In [ ]:
counts = train_df["label"].value_counts().to_dict()
N = len(train_df)
w0 = N / (2.0 * counts.get(0, 1))
w1 = N / (2.0 * counts.get(1, 1))
class_weights = torch.tensor([w0, w1], dtype=torch.float)

print("라벨 분포:", counts, "| class_weights:", class_weights.tolist())

# 모델 로드 (2 클래스)
id2label = {0: "non_toxic", 1: "toxic"}
label2id = {"non_toxic": 0, "toxic": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

# Trainer에서 쓸 가중치 손실 함수 래핑
from torch.nn import CrossEntropyLoss

def compute_weighted_loss(outputs, labels):
    # outputs.logits: [B,2], labels: [B]
    loss_fct = CrossEntropyLoss(weight=class_weights.to(outputs.logits.device))
    return loss_fct(outputs.logits, labels)

라벨 분포: {1: 5808, 0: 4320} | class_weights: [1.1722222566604614, 0.8719007968902588]


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
acc_metric  = evaluate.load("accuracy")
prec_metric = evaluate.load("precision")
rec_metric  = evaluate.load("recall")
f1_metric   = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy":  acc_metric.compute(predictions=preds, references=labels)["accuracy"],
        "precision": prec_metric.compute(predictions=preds, references=labels, average="binary")["precision"],
        "recall":    rec_metric.compute(predictions=preds, references=labels, average="binary")["recall"],
        "f1":        f1_metric.compute(predictions=preds, references=labels, average="binary")["f1"],
    }


In [ ]:
import torch, platform
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

CUDA: True
GPU: NVIDIA A100-SXM4-40GB


In [ ]:
BATCH = 8          # GPU 여유 없으면 8로
EPOCHS = 3          # 3~5 권장
LR = 2e-5

args = TrainingArguments(
    output_dir="/content/tox_fix_ckpt",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_steps=100,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**{k:v for k,v in inputs.items() if k != "labels"})
        loss_fct = CrossEntropyLoss(weight=class_weights.to(outputs.logits.device))
        loss = loss_fct(outputs.logits, labels)
        return (loss, outputs) if return_outputs else loss

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()


/tmp/ipython-input-1093762272.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.106900,0.158610,0.962875,0.953271,0.983471,0.968136
2,0.072300,0.166832,0.970379,0.966780,0.982094,0.974376
3,0.017800,0.201736,0.965245,0.972299,0.966942,0.969613


{'eval_loss': 0.16683246195316315,
 'eval_accuracy': 0.9703791469194313,
 'eval_precision': 0.9667796610169491,
 'eval_recall': 0.9820936639118457,
 'eval_f1': 0.9743764947044756,
 'eval_runtime': 3.0074,
 'eval_samples_per_second': 841.93,
 'eval_steps_per_second': 52.87,
 'epoch': 3.0}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 저장할 드라이브 경로 (원하는 폴더명으로 변경)
SAVE_DIR = "/content/drive/MyDrive/tox_roberta_best"

# 모델 + 토크나이저 저장
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

print("✅ 모델과 토크나이저가 구글 드라이브에 저장됨:", SAVE_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 모델과 토크나이저가 구글 드라이브에 저장됨: /content/drive/MyDrive/tox_roberta_best


In [ ]:
# ==== Toxic Inference with short-text & jamo rules ====
import re, torch, numpy as np
from typing import List, Union
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 설정(네 경로/값으로 맞춰)
INFER_DIR = "/content/drive/MyDrive/tox_roberta_best"
BADWORDS_FILE = "bad_words.txt"
THRESHOLD = 0.95
MAX_LEN = 128

device = "cuda" if torch.cuda.is_available() else "cpu"
tok = AutoTokenizer.from_pretrained(INFER_DIR)
md  = AutoModelForSequenceClassification.from_pretrained(INFER_DIR).to(device).eval()

# badwords 로드(없으면 규칙 skip)
BADWORDS_RE = None
try:
    pats = []
    with open(BADWORDS_FILE, "r", encoding="utf-8") as f:
        for line in f:
            raw = line.strip()
            if not raw or raw.startswith("#"):
                continue
            pats.append(raw[3:].strip() if raw.startswith("re:") else re.escape(raw))
    if pats:
        BADWORDS_RE = re.compile("(?:" + "|".join(pats) + ")", flags=re.IGNORECASE)
except FileNotFoundError:
    pass

def contains_badword(s: str) -> bool:
    return bool(BADWORDS_RE and isinstance(s,str) and BADWORDS_RE.search(s))

def mask_badwords(s: str, mask="***") -> str:
    return BADWORDS_RE.sub(mask, s) if (BADWORDS_RE and isinstance(s,str)) else s

# === 새 규칙용 헬퍼들 ===
_JAMO_ONLY_RE = re.compile(r"^[ㄱ-ㅎㅏ-ㅣ]+$")  # 자음/모음만
_VALID_CHARS_RE = re.compile(r"[^0-9A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ]+")

def is_jamo_only(s: str) -> bool:
    s = (s or "").strip()
    return bool(s) and bool(_JAMO_ONLY_RE.fullmatch(s))

def effective_len(s: str) -> int:
    """공백/기호 제거 후 유효 글자수(가-힣/자모/영문/숫자)"""
    s = (s or "").strip()
    s = _VALID_CHARS_RE.sub("", s)
    return len(s)

@torch.no_grad()
def predict_toxic(
    texts: Union[str, List[str]],
    threshold: float = THRESHOLD,
    force_badwords: bool = True,
    return_masked: bool = True,
):
    single = False
    if isinstance(texts, str):
        texts = [texts]
        single = True

    probs = np.zeros((len(texts),), dtype=float)
    preds = np.zeros((len(texts),), dtype=int)
    rule  = [""] * len(texts)

    # 1) 자모만 → non_toxic
    for i, t in enumerate(texts):
        if is_jamo_only(t):
            probs[i] = 0.0
            preds[i] = 0
            rule[i]  = "jamo_only_non_toxic"

    # 2) badwords → toxic (자모-only보다 우선?)
    # 요구사항: "욕설이면 무조건 toxic" 이므로 badwords가 있으면 항상 toxic이 우선
    for i, t in enumerate(texts):
        if preds[i] == 0 and rule[i] == "jamo_only_non_toxic":
            # 자모-only인데 badword는 사실상 없음. 그래도 안전하게 체크
            pass
        if force_badwords and contains_badword(t):
            probs[i] = 1.0
            preds[i] = 1
            rule[i]  = "badwords_forced_toxic"

    # 3) 글자수 ≤ 5 이고 badword 없음 → non_toxic
    for i, t in enumerate(texts):
        if rule[i]:  # 이미 결정된 경우 skip
            continue
        if effective_len(t) <= 5 and not contains_badword(t):
            probs[i] = 0.0
            preds[i] = 0
            rule[i]  = "short_text_non_toxic"

    # 4) 남은 것만 모델 추론
    idxs = [i for i, r in enumerate(rule) if r == ""]
    if idxs:
        batch = [texts[i] for i in idxs]
        enc = tok(batch, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN)
        enc = {k: v.to(device) for k, v in enc.items()}
        logits = md(**enc).logits
        p = torch.softmax(logits, dim=-1)[:, 1].cpu().numpy()  # toxic 확률
        for j, i in enumerate(idxs):
            probs[i] = float(p[j])
            preds[i] = int(p[j] >= threshold)
            rule[i]  = "model_threshold"

    # 결과 구성
    out = []
    for i, t in enumerate(texts):
        item = {
            "text": t,
            "prob_toxic": float(probs[i]),
            "pred_label": "toxic" if int(preds[i]) == 1 else "non_toxic",
            "rule": rule[i],
            "threshold": float(threshold),
        }
        if return_masked:
            item["masked_text"] = mask_badwords(t)
        out.append(item)

    return out[0] if single else out

# 사용 예시
tests = [
    "ㅋㅋㅋㅋ",         # 자모-only → non_toxic
    "시발",             # badword → toxic
    "뭐함?",            # 3글자 & badword X → non_toxic
    "몰랐는데 영상에 세마리잖아",  # 모델 판단
]
for r in predict_toxic(tests):
    print(r)


{'text': 'ㅋㅋㅋㅋ', 'prob_toxic': 0.0, 'pred_label': 'non_toxic', 'rule': 'jamo_only_non_toxic', 'threshold': 0.95, 'masked_text': 'ㅋㅋㅋㅋ'}
{'text': '시발', 'prob_toxic': 1.0, 'pred_label': 'toxic', 'rule': 'badwords_forced_toxic', 'threshold': 0.95, 'masked_text': '***'}
{'text': '뭐함?', 'prob_toxic': 0.0, 'pred_label': 'non_toxic', 'rule': 'short_text_non_toxic', 'threshold': 0.95, 'masked_text': '뭐함?'}
{'text': '몰랐는데 영상에 세마리잖아', 'prob_toxic': 0.9926854372024536, 'pred_label': 'toxic', 'rule': 'model_threshold', 'threshold': 0.95, 'masked_text': '몰랐는데 영상에 세마리잖아'}


In [ ]:
import torch, numpy as np

pred_output = trainer.predict(valid_ds)
logits = pred_output.predictions
labels = pred_output.label_ids
probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()
prob_toxic = probs[:, 1]  # toxic 확률
y_true = labels

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score

def eval_at_threshold(t):
    y_pred = (prob_toxic >= t).astype(int)
    return {
        "thr": t,
        "acc": accuracy_score(y_true, y_pred),
        "prec": precision_score(y_true, y_pred),
        "rec": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
    }

# 0.50 / 0.95 / 최적 thr 비교
scan = np.linspace(0.05, 0.99, 95)
best = max((eval_at_threshold(t) for t in scan), key=lambda d: d["f1"])

print("AUC-ROC  :", roc_auc_score(y_true, prob_toxic))
print("AUC-PR   :", average_precision_score(y_true, prob_toxic))
print("t=0.50   :", eval_at_threshold(0.50))
print("t=0.95   :", eval_at_threshold(0.95))
print("best_thr :", best)

AUC-ROC  : 0.9921053974084277
AUC-PR   : 0.9933498236058635
t=0.50   : {'thr': 0.5, 'acc': 0.9703791469194313, 'prec': 0.9667796610169491, 'rec': 0.9820936639118457, 'f1': 0.9743764947044756}
t=0.95   : {'thr': 0.95, 'acc': 0.9699842022116903, 'prec': 0.9699453551912568, 'rec': 0.977961432506887, 'f1': 0.9739368998628258}
best_thr : {'thr': np.float64(0.13), 'acc': 0.9707740916271722, 'prec': 0.9655405405405405, 'rec': 0.9841597796143251, 'f1': 0.9747612551159618}


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

THRESHOLD = 0.95  # 네가 운영에 쓰려는 값으로
y_pred = (prob_toxic >= THRESHOLD).astype(int)

print("Confusion matrix @ thr=%.2f" % THRESHOLD)
print(confusion_matrix(y_true, y_pred))
print("\nReport @ thr=%.2f" % THRESHOLD)
print(classification_report(y_true, y_pred, target_names=["non_toxic","toxic"], digits=4))

Confusion matrix @ thr=0.95
[[1036   44]
 [  32 1420]]

Report @ thr=0.95
              precision    recall  f1-score   support

   non_toxic     0.9700    0.9593    0.9646      1080
       toxic     0.9699    0.9780    0.9739      1452

    accuracy                         0.9700      2532
   macro avg     0.9700    0.9686    0.9693      2532
weighted avg     0.9700    0.9700    0.9700      2532



In [ ]:
import json, pandas as pd
from pathlib import Path

AUG_PATH = Path("/content/drive/MyDrive/tox_roberta_best/augmented_error_samples.jsonl")

# JSONL -> DataFrame
aug_rows = []
with AUG_PATH.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        aug_rows.append(json.loads(line))
aug_df = pd.DataFrame(aug_rows)
assert {"message","label"}.issubset(aug_df.columns), aug_df.columns
print("증강 데이터:", aug_df.shape)
aug_df.head(3)

증강 데이터: (119, 2)


,message,label
0,멍청이 수사 웃기네 시작됐네. 흐지부지 각!,1
1,수준 보인다 수사 또 바보 흐지부지 각,1
2,진짜 개 자기가 무시한 거니까 어쩔 수 없지,1


In [ ]:
def normalize_label(x):
    if isinstance(x, str):
        xs = x.strip().lower()
        if xs in ["toxic","1","true","yes","y"]:
            return 1
        if xs in ["non_toxic","0","false","no","n","neutral","clean"]:
            return 0
    if isinstance(x, (int, float)):
        return 1 if int(x)==1 else 0
    return 0

aug_df["label"] = aug_df["label"].apply(normalize_label).astype(int)

# 같은 문장 중복 제거(있으면)
before = len(aug_df)
aug_df = aug_df.drop_duplicates(subset=["message"]).reset_index(drop=True)
print(f"중복 제거: {before} -> {len(aug_df)}")
aug_df.head(3)

중복 제거: 119 -> 119


,message,label
0,멍청이 수사 웃기네 시작됐네. 흐지부지 각!,1
1,수준 보인다 수사 또 바보 흐지부지 각,1
2,진짜 개 자기가 무시한 거니까 어쩔 수 없지,1


In [ ]:
import pandas as pd
from pathlib import Path

# 케이스 A) 이미 메모리에 있는 경우(권장)
try:
    train_df, valid_df
except NameError:
    # 케이스 B) 파일에서 불러오는 경우 (네 경로로 바꿔)
    TRAIN_ORIG = Path("/content/train_messages.jsonl")
    VALID_ORIG = Path("/content/valid_messages.jsonl")

    def load_jsonl(p: Path):
        rows = []
        with p.open("r", encoding="utf-8") as f:
            for line in f:
                line=line.strip()
                if not line: continue
                rows.append(json.loads(line))
        return pd.DataFrame(rows)

    train_df = load_jsonl(TRAIN_ORIG)
    valid_df = load_jsonl(VALID_ORIG)

# 컬럼 확인
assert {"message","label"}.issubset(train_df.columns), train_df.columns
assert {"message","label"}.issubset(valid_df.columns), valid_df.columns

# 병합 (증강만 train에 추가, 검증셋은 유지)
merged_train = pd.concat([train_df[["message","label"]], aug_df[["message","label"]]], axis=0, ignore_index=True)

# 최종 중복 제거 + 섞기
before = len(merged_train)
merged_train = merged_train.drop_duplicates(subset=["message"]).sample(frac=1.0, random_state=42).reset_index(drop=True)
print(f"병합 완료: {before} -> {len(merged_train)} (train), valid={len(valid_df)}")
merged_train.head(3)

병합 완료: 10247 -> 9355 (train), valid=2532


,message,label
0,"아 진짜, 이 정도면 사람 아닌 수준이다 정신 좀 차려",1
1,넌 물놀이가 아니라 몸매놀이 하러 가는 거잖아ㅋㅋ,1
2,넌 스스로 거리두기 하잖아ㅋㅋ,1


In [ ]:
import numpy as np, torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

# 이전에 쓰던 설정과 동일하게 맞춰
MODEL_NAME = "klue/roberta-small"   # 네가 썼던 모델로(예: klue/roberta-base / small)
MAX_LEN = 128                       # 학습 때 썼던 값으로

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LEN)

train_ds = Dataset.from_pandas(merged_train.rename(columns={"message":"text","label":"labels"}), preserve_index=False)
valid_ds = Dataset.from_pandas(valid_df.rename(columns={"message":"text","label":"labels"}), preserve_index=False)

train_ds = train_ds.map(lambda b: tokenize_fn(b), batched=True, remove_columns=["text"])
valid_ds = valid_ds.map(lambda b: tokenize_fn(b), batched=True, remove_columns=["text"])

keep = ["input_ids","attention_mask","labels"]
train_ds = train_ds.remove_columns([c for c in train_ds.column_names if c not in keep])
valid_ds = valid_ds.remove_columns([c for c in valid_ds.column_names if c not in keep])

datasets = DatasetDict(train=train_ds, validation=valid_ds)
datasets

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

Map:   0%|          | 0/9355 [00:00<?, ? examples/s]

Map:   0%|          | 0/2532 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 9355
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2532
    })
})

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

# 이전에 fine-tune해서 드라이브에 저장해둔 체크포인트에서 이어서 시작
# (없으면 MODEL_NAME에서 새로 시작)
CKPT = "/content/drive/MyDrive/tox_roberta_best"  # 네가 저장한 모델 폴더

try:
    model = AutoModelForSequenceClassification.from_pretrained(
        CKPT, num_labels=2, id2label={0:"non_toxic",1:"toxic"}, label2id={"non_toxic":0,"toxic":1}
    )
    print("✅ Loaded from previous checkpoint:", CKPT)
except Exception as e:
    print("⚠️ Fallback to base model:", e)
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, num_labels=2, id2label={0:"non_toxic",1:"toxic"}, label2id={"non_toxic":0,"toxic":1}
    )

# 지표
acc = evaluate.load("accuracy"); prec = evaluate.load("precision")
rec = evaluate.load("recall"); f1 = evaluate.load("f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy":  acc.compute(predictions=preds, references=labels)["accuracy"],
        "precision": prec.compute(predictions=preds, references=labels, average="binary")["precision"],
        "recall":    rec.compute(predictions=preds, references=labels, average="binary")["recall"],
        "f1":        f1.compute(predictions=preds, references=labels, average="binary")["f1"],
    }

# 학습 세팅 (추가학습이므로 에폭/러닝레이트는 보수적으로)
BATCH = 32 if torch.cuda.is_available() else 16
args = TrainingArguments(
    output_dir="/content/tox_cont_ckpt",
    learning_rate=1e-5,                 # 기존보다 낮게
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    num_train_epochs=2,                 # 1~2 에폭 짧게
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_steps=200,
    fp16=torch.cuda.is_available(),
    group_by_length=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=tokenizer,   # 최신버전에선 processing_class 써도 OK
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

✅ Loaded from previous checkpoint: /content/drive/MyDrive/tox_roberta_best


/tmp/ipython-input-2302961708.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.044700,0.133088,0.970774,0.977809,0.971074,0.974430
2,0.030700,0.147644,0.969589,0.985180,0.961433,0.973161


{'eval_loss': 0.1330876648426056,
 'eval_accuracy': 0.9707740916271722,
 'eval_precision': 0.9778085991678225,
 'eval_recall': 0.9710743801652892,
 'eval_f1': 0.9744298548721493,
 'eval_runtime': 1.6057,
 'eval_samples_per_second': 1576.839,
 'eval_steps_per_second': 49.821,
 'epoch': 2.0}

In [ ]:
from google.colab import drive
from transformers import AutoTokenizer
drive.mount('/content/drive')

SAVE_DIR = "/content/drive/MyDrive/tox_roberta_best"  # 덮어쓰기(원치 않으면 다른 폴더명)
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
print("✅ 저장 완료:", SAVE_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 저장 완료: /content/drive/MyDrive/tox_roberta_best
